In [85]:
import pandas as pd
import numpy as np
import re

In [86]:
bancos = [
    'b1202304001',
    'b1202304037',
    'b1202304016',
    'b1202304012',
    'b1202304014'
]

In [87]:
dfs = {} # Diccionario de dataframes

for banco in bancos:
    ruta = f'./PruebaPM/{banco}.txt' # Ruta del archivo CSV
    df = pd.read_csv(ruta, sep="\t", header=None, names=range(5), encoding='latin1') # Leer el archivo CSV sin el header
    df['Archivo'] = banco
    df['Banco'] = df.iloc[0,0]
    df = df.iloc[1:, :]

    dfs[banco] = df # Agregar el dataframe al diccionario

df_final = pd.concat(dfs.values(), ignore_index=True) # Concatenar los dataframes

In [88]:
df_names = {0:'Cuenta_contable', 1:'Saldo_CLP',2:'Saldo_UF',3:'Saldo_USD',4:'Saldo_otras_monedas', 'Archivo':'Archivo', 'Banco':'Banco'}
df_final.rename(columns=df_names, inplace=True)

df_final = df_final.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_final.iloc[:, :4] = df_final.iloc[:, :4].replace(',', '.', regex=True).astype(float)
df_final.iloc[:, :4] = df_final.iloc[:, :4].astype(float)

df_final.head()

,Cuenta_contable,Saldo_CLP,Saldo_UF,Saldo_USD,Saldo_otras_monedas,Archivo,Banco
0,100000000.0,28444728250232.0,20128453882100.0,117300798857.0,"6694297877237,00",b1202304001,001 BANCO DE CHILE
1,105000000.0,1059628254454.0,0.0,0.0,"2016638137886,00",b1202304001,001 BANCO DE CHILE
2,105000100.0,823187780235.0,0.0,0.0,"0059105027666,00",b1202304001,001 BANCO DE CHILE
3,105000101.0,192860234135.0,0.0,0.0,"0059105027666,00",b1202304001,001 BANCO DE CHILE
4,105000102.0,630327546100.0,0.0,0.0,"0000000000000,00",b1202304001,001 BANCO DE CHILE


In [89]:
df_final['Periodo'] = df_final['Archivo'].str.extract(r'b1(\d{6})') #Extrae el periodo de la fecha de archivo
df_final['Banco'] = df_final['Banco'].str.extract(r'\b (.*)') # Extrae todo despues del primer espacio

# convertir periodo a tipo fecha
df_final['Periodo'] = pd.to_datetime(df_final['Periodo'], format='%Y%m')
df_final['Periodo'] = df_final['Periodo'].dt.strftime('%Y-%m')

# quitar columna Archivo
df_final = df_final.drop(df_final.columns[5], axis=1)

In [90]:
df_final.head(10)

,Cuenta_contable,Saldo_CLP,Saldo_UF,Saldo_USD,Saldo_otras_monedas,Banco,Periodo
0,100000000.0,28444728250232.0,20128453882100.0,117300798857.0,"6694297877237,00",BANCO DE CHILE,2023-04
1,105000000.0,1059628254454.0,0.0,0.0,"2016638137886,00",BANCO DE CHILE,2023-04
2,105000100.0,823187780235.0,0.0,0.0,"0059105027666,00",BANCO DE CHILE,2023-04
3,105000101.0,192860234135.0,0.0,0.0,"0059105027666,00",BANCO DE CHILE,2023-04
4,105000102.0,630327546100.0,0.0,0.0,"0000000000000,00",BANCO DE CHILE,2023-04
5,105000103.0,0.0,0.0,0.0,"0000000000000,00",BANCO DE CHILE,2023-04
6,105000200.0,230119512360.0,0.0,0.0,"0350945711497,00",BANCO DE CHILE,2023-04
7,105000201.0,230119512360.0,0.0,0.0,"0350945711497,00",BANCO DE CHILE,2023-04
8,105000202.0,0.0,0.0,0.0,"0000000000000,00",BANCO DE CHILE,2023-04
9,105000209.0,0.0,0.0,0.0,"0000000000000,00",BANCO DE CHILE,2023-04


In [91]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3545 entries, 0 to 3544
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Cuenta_contable      3545 non-null   object
 1   Saldo_CLP            3545 non-null   object
 2   Saldo_UF             3545 non-null   object
 3   Saldo_USD            3545 non-null   object
 4   Saldo_otras_monedas  3545 non-null   object
 5   Banco                3545 non-null   object
 6   Periodo              3545 non-null   object
dtypes: object(7)
memory usage: 194.0+ KB


In [92]:
df_final.to_csv('final_df.csv', index=False)